<a href="https://colab.research.google.com/github/pravsels/DistilLM/blob/main/HF_Training_Script2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers accelerate peft bitsandbytes

In [2]:
! apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
import transformers
from datasets import load_dataset
import math
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

In [5]:
wikitext_dataset = load_dataset('mediciresearch/manimation')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
wikitext_dataset['train'][0]

{'text': '<s>[INST] Generate accurate and correct ManimCE Python code for the animation requested by the user. Here is the user\'s request: Could you create a 3D animated video using ManimCE where a cylinder appears and then fades out, followed by a drawing of a 3D coordinate system, and finally my name "Kiron" appears above the coordinate system? The camera should give us an angled aerial view of the scene. The appearance of the cylinder should take 3 seconds. [/INST] \\n from manim import *\n\nclass AreaUnderCurveExample(Scene):\n    def construct(self):\n        # Displaying the problem\n        problem = MathTex(r"\\int_1^3 x^2 \\,dx")\n        self.play(Write(problem))\n        self.wait(1)\n\n        # Explanation for setting up the integral\n        explanation1 = Text("Find the area under x^2 from 1 to 3", font_size=24).to_edge(UP)\n        self.play(Write(explanation1))\n        self.wait(2)\n\n        # Integrating the function\n        integration_step = MathTex(r"= \\left[\

In [7]:
model_name = 'deepseek-ai/deepseek-coder-6.7b-instruct'
# model_name = 'smallcloudai/Refact-1_6B-fim'
# using the same tokenizer that was used for training the model
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [8]:
## load a dataset with prompts and such for manim
# note in our case we use only the code witout the prompts
from datasets import load_dataset
data_files = {"train": "train.csv", "test": "test.csv"}
dataset = load_dataset("Edoh/manim_python", data_files=data_files)
#dataset.rename_column("test", "new_column_name")
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 599
    })
    test: Dataset({
        features: ['instruction', 'output'],
        num_rows: 51
    })
})


In [9]:
def tokenize_func(examples):
  return tokenizer(examples['output'])

In [10]:
tokenized_dataset = dataset.map(tokenize_func,
                                         batched=True,
                                         num_proc=8,
                                         remove_columns=['instruction', 'output'])




In [11]:
print(tokenized_dataset)
print(tokenized_dataset['train'][5])

# features: ['input_ids', 'attention_mask', 'labels'],

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 599
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 51
    })
})
{'input_ids': [32013, 3154, 664, 308, 1659, 572, 757, 2669, 3987, 3033, 7, 3987, 3033, 1772, 972, 5580, 7, 1180, 1772, 7748, 405, 18844, 7, 2716, 62, 4082, 28, 18, 11, 3032, 28, 18851, 8, 1781, 13, 1761, 7, 19497, 8, 1781, 13, 2035, 7, 19497, 13, 261, 3357, 13, 6708, 7, 17, 650, 1404, 62, 2915, 28, 16, 13, 20, 8], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [12]:
# setting context length of out inputs
# block_size = tokenizer.model_max_length
block_size = 1024
    # context len of input

In [41]:
def group_text_func(examples):
  # for every key ('train', 'test' etc.), concatenate the texts
  concat_examples = {key: sum(examples[key], []) for key in examples.keys()}
  total_len = len(concat_examples[list(examples.keys())[0]])
  total_len = (total_len // block_size) * block_size

  # split by chunks of block_size
  result = {
      key: [text[i : i + block_size] for i in range(0, total_len, block_size)] for key, text in concat_examples.items()
  }
  result['labels'] = result['input_ids'].copy()

  return result

In [43]:
lm_dataset = tokenized_dataset.map(group_text_func,
                                   batched=True,
                                   batch_size=512,
                                   num_proc=8)

#lm_dataset = tokenized_dataset
lm_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 32
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 0
    })
})

In [38]:
print('input ids : ', lm_dataset['train'][0]['input_ids'][:5])
print('attention mask : ', lm_dataset['train'][0]['attention_mask'][:5])
print('label ids : ', lm_dataset['train'][0]['labels'][:5])
#print(i != None for i in lm_dataset['train'][i] for i in range(0, ))
#print('label ids : ', lm_dataset['train'][0])
print(lm_dataset)

input ids :  [32013, 3154, 664, 308, 1659]
attention mask :  [1, 1, 1, 1, 1]
label ids :  [32013, 3154, 664, 308, 1659]
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 32
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 0
    })
})
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 32
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 0
    })
})


In [16]:
from peft import LoraConfig

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj"],
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [17]:

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             load_in_8bit=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
model.add_adapter(peft_config,
                  adapter_name="manim_adapter")

In [19]:
import numpy as np
def count_trainable_parameters(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    total_params = sum([np.prod(p.size()) for p in model.parameters()])
    return params/total_params

count_trainable_parameters(model) * 100

0.49533656825843125

In [20]:
training_args = TrainingArguments(
  f"{model_name}-finetuned-manimation",
  evaluation_strategy = "epoch",
  learning_rate=2e-5,
  per_device_train_batch_size=1,
  per_device_eval_batch_size=1,
  gradient_accumulation_steps=1,
  weight_decay=0.01,
  push_to_hub=False,
  do_eval=False
)

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset['train'],
    eval_dataset=lm_dataset['train']
)

In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.427808
2,No log,0.393158
3,No log,0.379410


TrainOutput(global_step=96, training_loss=0.41291054089864093, metrics={'train_runtime': 94.3567, 'train_samples_per_second': 1.017, 'train_steps_per_second': 1.017, 'total_flos': 3917579525554176.0, 'train_loss': 0.41291054089864093, 'epoch': 3.0})

In [23]:
eval_results = trainer.evaluate()
print(f"Perplexity score : {math.exp(eval_results['eval_loss']):.2f}")

Perplexity score : 1.46


In [24]:
gen_context_len = 4096

input_text = "write manim code to transition a blue circle to a red square."
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_length=gen_context_len)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


write manim code to transition a blue circle to a red square.




A: You can use the `UpdateFromConfig` method to change the color of the circle to red and the size of the square to match the circle.

Here is an example:

```python
from manim import *

class CircleToSquare(Scene):
    def construct(self):
        circle = Circle(color=BLUE)
        square = Square(color=RED)

        self.play(Create(circle))
        self.play(Transform(circle, square))
        self.wait()
```

In this example, the circle is created first, then it is transformed into a square. The `Transform` method is used to smoothly transition from one shape to another.

If you want to change the color of the square to match the circle, you can use the `UpdateFromConfig` method:

```python
from manim import *

class CircleToSquare(Scene):
    def construct(self):
        circle = Circle(color=BLUE)
        square = Square(color=RED)

        self.play(Create(circle))
        self.play(Transform(circle, square))
    

In [25]:
#trainer.push_to_hub()

In [26]:
# load_model_hub = AutoModelForCausalLM.from_pretrained("pravsels/distilgpt2-finetuned-wikitext2")

In [27]:
# from transformers import DataCollatorForLanguageModeling, AutoModelForMaskedLM

# # masked language modelling
# model_name = "distilroberta-base"

# tokenizer = AutoTokenizer.from_pretrained(model_name,
#                                           use_fast=True)

# tokenized_dataset = wikitext_dataset.map(tokenize_func,
#                                          batched=True,
#                                          num_proc=8,
#                                          remove_columns=["text"])



# lm_dataset = tokenized_dataset.map(
#     group_text_func,
#     batched=True,
#     batch_size=512,
#     num_proc=8
# )

# model = AutoModelForMaskedLM.from_pretrained(model_name)

# training_args = TrainingArguments(
#     f"{model_name}-finetuned-wikitext2",
#     evaluation_strategy = "epoch",
#     learning_rate=2e-5,
#     weight_decay=0.01,
#     push_to_hub=True,
# )

# # we do this for random masking of tokens
# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
#                                                 mlm_probability=0.15)


# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=lm_dataset["train"],
#     eval_dataset=lm_dataset["validation"],
#     data_collator=data_collator,
# )

# trainer.train()

In [28]:
# eval_results = trainer.evaluate()
# print(f"Perplexity score : {math.exp(eval_results['eval_loss']):.2f}")

In [29]:
# trainer.push_to_hub()